In [1]:
import pandas as pd
import numpy as np
import os
import cv2
import tensorflow as tf
import glob
import xml.etree.ElementTree as ET
import h5py
import warnings


# from data_generator.object_detection_2d_data_generator import DataGenerator

In [2]:
warnings.filterwarnings('ignore')

#входное разрешение модели
IMAGE_HEIGHT = 28 # Height of the input images
IMAGE_WIDTH = 28 #Width of the input images
WORK_DIR = 'D:\work\\test_comp_vision\datasets\!_lines_w25_parsed_test' #путь к основной папке

In [18]:
"""
Структура папок:
base_folder
    image_0
        image_0_0.jpg
        image_0_0.txt
        image_0_1.jpg
        image_0_1.txt
        ...
    image_1
        image_1_0.jpg
        image_1_0.txt
        image_1_1.jpg
        image_1_1.txt
        ...
        
"""

def get_dataframe(path):
    symbol_classes = []
    for folder in os.listdir(path):
        print(folder)
        for txt_file in glob.glob(f'{path}/{folder}/*.txt'):
            print(txt_file)
            with open(txt_file, 'r') as f:
                symbol_class = f.read()
            value = (txt_file, symbol_class)
            symbol_classes.append(value)
    column_names = ['file_path', 'class']
    return pd.DataFrame(symbol_classes, columns=column_names)

In [19]:
#Создаем DataFrame из всех имеющихся изображений
base_path = WORK_DIR #путь к основной папке
df = pd.DataFrame()
#for obj_folder in os.listdir(base_path):
#    df = df.append(get_dataframe(os.path.join(base_path, obj_folder)))
df = df.append(get_dataframe(base_path))
df.head()

0_a
D:\work\test_comp_vision\datasets\!_lines_w25_parsed_test/0_a\0_a_0.txt
D:\work\test_comp_vision\datasets\!_lines_w25_parsed_test/0_a\0_a_1.txt
D:\work\test_comp_vision\datasets\!_lines_w25_parsed_test/0_a\0_a_10.txt
D:\work\test_comp_vision\datasets\!_lines_w25_parsed_test/0_a\0_a_11.txt
D:\work\test_comp_vision\datasets\!_lines_w25_parsed_test/0_a\0_a_12.txt
D:\work\test_comp_vision\datasets\!_lines_w25_parsed_test/0_a\0_a_13.txt
D:\work\test_comp_vision\datasets\!_lines_w25_parsed_test/0_a\0_a_14.txt
D:\work\test_comp_vision\datasets\!_lines_w25_parsed_test/0_a\0_a_15.txt
D:\work\test_comp_vision\datasets\!_lines_w25_parsed_test/0_a\0_a_16.txt
D:\work\test_comp_vision\datasets\!_lines_w25_parsed_test/0_a\0_a_17.txt
D:\work\test_comp_vision\datasets\!_lines_w25_parsed_test/0_a\0_a_18.txt
D:\work\test_comp_vision\datasets\!_lines_w25_parsed_test/0_a\0_a_2.txt
D:\work\test_comp_vision\datasets\!_lines_w25_parsed_test/0_a\0_a_3.txt
D:\work\test_comp_vision\datasets\!_lines_w25_parse

,file_path,class
0,D:\work\test_comp_vision\datasets\!_lines_w25_...,в
1,D:\work\test_comp_vision\datasets\!_lines_w25_...,е
2,D:\work\test_comp_vision\datasets\!_lines_w25_...,е
3,D:\work\test_comp_vision\datasets\!_lines_w25_...,р
4,D:\work\test_comp_vision\datasets\!_lines_w25_...,а


In [18]:
df

,file_path,class


In [10]:
#Преобразуем координаты рамок к нужному размеру
labels_to_int = {'person':1, 'crane':2}
# df = df[df['class'] == 'car']
df = pd.DataFrame({'image_name':df['filename'],
                   'xmin':(df['xmin']/df['width']*img_width).astype(int),
                   'xmax':(df['xmax']/df['width']*img_width).astype(int),
                   'ymin':(df['ymin']/df['height']*img_height).astype(int),
                   'ymax':(df['ymax']/df['height']*img_height).astype(int),
                   'class_id':[labels_to_int[label] for label in df['class']]})

df.to_csv('data/ZAVOD.csv', index=False)
df

,image_name,xmin,xmax,ymin,ymax,class_id
0,D:/Program Files/ML-NN/traind_ssd/DATASET\ZAVO...,212,236,116,141,1
1,D:/Program Files/ML-NN/traind_ssd/DATASET\ZAVO...,426,512,11,61,2
2,D:/Program Files/ML-NN/traind_ssd/DATASET\ZAVO...,213,237,116,141,1
3,D:/Program Files/ML-NN/traind_ssd/DATASET\ZAVO...,426,512,11,61,2
4,D:/Program Files/ML-NN/traind_ssd/DATASET\ZAVO...,213,236,116,141,1
...,...,...,...,...,...,...
12417,D:/Program Files/ML-NN/traind_ssd/DATASET\ZAVO...,424,512,12,68,2
12418,D:/Program Files/ML-NN/traind_ssd/DATASET\ZAVO...,293,314,115,164,1
12419,D:/Program Files/ML-NN/traind_ssd/DATASET\ZAVO...,281,290,67,91,1
12420,D:/Program Files/ML-NN/traind_ssd/DATASET\ZAVO...,461,491,230,272,1


In [11]:
len(pd.unique(df['image_name']))

2706

In [12]:
#Создаем файл датасета для обучения детектора
train_dataset = DataGenerator(load_images_into_memory=False, hdf5_dataset_path=None)
images_dir = ''

train_labels_filename = 'data/ZAVOD.csv'

train_dataset.parse_csv(images_dir=images_dir,
                        labels_filename=train_labels_filename,
                        input_format=['image_name', 'xmin', 'xmax', 'ymin', 'ymax', 'class_id'], # This is the order of the first six columns in the CSV file that contains the labels for your dataset. If your labels are in XML format, maybe the XML parser will be helpful, check the documentation.
                        include_classes='all')

train_dataset.create_hdf5_dataset(file_path='data/dataset_zavod_288x512.h5',
                                  resize=[img_height,img_width],
                                  variable_image_size=True,
                                  verbose=True)

Creating HDF5 dataset: 100%|██████████| 2706/2706 [00:48<00:00, 55.91it/s]
